In [ ]:
import numpy as np
import qiskit as q
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ, QuantumRegister, AncillaRegister
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import Statevector
from qiskit.visualization import array_to_latex
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  
import matplotlib.cm as cm
import qiskit.circuit.library as qlib
%matplotlib inline
from qiskit import transpile
import os

In [ ]:

Nx = 16       
Ny = 16       
Q = 5
L = Nx-1
dt=1.
dim = 2
dirs = 5

nlatx  = int(np.ceil(np.log2(Nx)))
nlaty  = int(np.ceil(np.log2(Ny)))
nlinks = 3

start_latx = 0
start_laty = start_latx + nlatx
start_links = start_laty + nlaty

nlatx1 = list(range(start_latx, start_laty))
nlaty1 = list(range(start_laty, start_links))
nlinks1 = list(range(start_links, start_links + nlinks))

Qexlink= nlatx+nlaty+1

rho = np.ones ((Ny, Nx)) 
vor = np.zeros ((Ny, Nx)) 
phi = np.zeros ((Ny, Nx)) 
ux  = np.zeros((Ny, Nx))
uy  = np.zeros((Ny, Nx))
dd  = np.zeros((Ny, Nx))
dudx= np.zeros((Ny, Nx))
dudy= np.zeros((Ny, Nx))
dvdx= np.zeros((Ny, Nx))
dvdy= np.zeros((Ny, Nx))
dvordx= np.zeros((Ny, Nx))
dphidy= np.zeros((Ny, Nx))
Ux  = np.zeros((Ny, Nx))
Uy  = np.zeros((Ny, Nx))
T   = np.zeros((Ny, Nx))
phii   = np.zeros((Ny, Nx))

dTdx = np.zeros((Ny,Nx))
dTdy = np.zeros((Ny,Nx))
Nu  = np.zeros((Ny, Nx))

In [ ]:
Pr = 0.71
Ra = 1000
Re=30
U = 0.1
cs2=1/3
cvis=U*L/Re
cdif=cvis/Pr

tau = 3*U*L/Re + 0.5
taut = 3*U*L/Re/Pr+0.5


In [ ]:

def top_half(A):
    def convert(val):
        return val+1j*np.sqrt(1-val**2)
    func = np.vectorize(convert)
    return func(A)
def bottom_half(A):
    def convert(val):
        return val-1j*np.sqrt(1-val**2)
    func = np.vectorize(convert)
    return func(A)

In [ ]:

def rshift(n):
    circ = QuantumCircuit(n)
    for i in range(n):
        if i == n-1:
            circ.x(i)
        else:
            circ.mcx(list(range(i+1,n)), i)
    return circ

def lshift(n):
    circ = QuantumCircuit(n)
    for i in reversed(range(n)):
        if i == n-1:
            circ.x(i)
        else:
            circ.mcx(list(range(i+1,n)), i)
    return circ

In [ ]:

def UVbounds():

    ux[ 0, :] =0
    uy[ 0, :] =0    

    ux[-1, :] =0
    uy[-1, :] =0

    ux[:,  0] = 0 
    uy[:,  0] = 0
    
    ux[:, -1] = 0
    uy[:, -1] = 0

    return ux,uy

def pvbounds(phi):

    phi[ 0, :] =0
    phi[-1, :] =0
    vor[ 0, :] =-2.*phi[1,:]
    vor[-1, :] =-2.*phi[-2,:]

    phi[:,  0] = 0    
    phi[:, -1] = 0
    vor[:,  0] = -2.*phi[:, 1]   
    vor[:, -1] = -2.*phi[:,-2]

    return phi,vor

def Tbounds():

    T[ 0, :] =T[ 1, :]
    T[-1, :] =T[-2, :]
 
    T[:,  0] = 1
    T[:, -1] = 0

    return T

def graduv(vor,T):

    dvordy = (vor[2:, 1:-1] - vor[:-2, 1:-1])/2/dt
    dvordx = (vor[1:-1, 2:] - vor[1:-1, :-2])/2/dt  

    dvordx = np.pad(dvordx, ((1, 1), (1, 1)), mode='edge')  
    dvordy = np.pad(dvordy, ((1, 1), (1, 1)), mode='edge')      

    dvordx[:, -1] = dvordx[:, -2]
    dvordx[:,  0] = dvordx[:,  1]

    dvordy[ 0, :] = dvordy[ 1, :]   
    dvordy[-1, :] = dvordy[-1, :]



    dTdy = (T[2:, 1:-1] - T[:-2, 1:-1])/2/dt
    dTdx = (T[1:-1, 2:] - T[1:-1, :-2])/2/dt  

    dTdx = np.pad(dTdx, ((1, 1), (1, 1)), mode='edge')  
    dTdy = np.pad(dTdy, ((1, 1), (1, 1)), mode='edge')    

    dTdx[:,  0] =dTdx[:,  1]     
    dTdx[:, -1] =dTdx[:, -2] 

    dTdy[ 0, :] =0
    dTdy[-1, :] =0

    return dvordx,dvordy,dTdx,dTdy

    
def vorf(rho,vor,ux,uy,dvordx,dvordy):   
    f    = np.zeros((Q, Ny, Nx))
    fneq = np.zeros((Q, Ny, Nx)) 
    feq  = np.zeros((Q, Ny, Nx)) 

    ex = np.array([0, 1, -1, 0,  0], dtype=np.int_)
    ey = np.array([0, 0,  0, 1, -1], dtype=np.int_)
    wt = np.array([2/6, 1/6, 1/6, 1/6, 1/6])   

    ex_3d = ex.reshape(Q, 1, 1)
    ey_3d = ey.reshape(Q, 1, 1)
    wt_3d = wt.reshape(Q, 1, 1)   

    eu   =  ex_3d * ux[np.newaxis, :, :] + ey_3d * uy[np.newaxis, :, :]
    eed  = (ex_3d - ux[np.newaxis, :, :]) * dvordx[np.newaxis, :, :] + (ey_3d - uy[np.newaxis, :, :]) * dvordy[np.newaxis, :, :]

    feq [:, :, :] = wt_3d*vor[np.newaxis, :, :] * (1 + eu/cs2)    
    f   [:, :, :] = feq[:, :, :]*(1-(tau-1.)*dt*eed/cs2)
    fneq[:, :, :] = f[:, :, :] - feq[:, :, :]

    f[3, 0, :]  = feq[3,  0, :]+fneq[3,  1, :]
    f[4,-1, :]  = feq[4, -1, :]+fneq[4, -2, :]
    f[1, :, 0]  = feq[1,  :, 0]+fneq[1,  :, 1] 
    f[2, :,-1]  = feq[2,  :,-1]+fneq[2,  :,-2]       

    return f.flatten() 

In [ ]:
def phif(rho,phi,ux,uy):   
    f    = np.zeros((Q, Ny, Nx))
    fneq = np.zeros((Q, Ny, Nx)) 
    feq  = np.zeros((Q, Ny, Nx)) 

    wt = np.array([2/6, 1/6, 1/6, 1/6, 1/6])   

    wt_3d = wt.reshape(Q, 1, 1)    
        
    feq [:, :, :] = wt_3d*phi[np.newaxis, :, :]   
    f   [:, :, :] = feq[:, :, :]
    fneq[:, :, :] = f[:, :, :] - feq[:, :, :]

    f[3, 0, :]  = feq[3,  0, :]+fneq[3,  1, :]
    f[4,-1, :]  = feq[4, -1, :]+fneq[4, -2, :]
    f[1, :, 0]  = feq[1,  :, 0]+fneq[1,  :, 1] 
    f[2, :,-1]  = feq[2,  :,-1]+fneq[2,  :,-2]       

    return f.flatten() 

In [ ]:
def Tf(rho,T,ux,uy,dTdx,dTdy): 
    f    = np.zeros((Q, Ny, Nx))
    fneq = np.zeros((Q, Ny, Nx)) 
    feq  = np.zeros((Q, Ny, Nx)) 

    ex = np.array([0, 1, -1, 0,  0], dtype=np.int_)
    ey = np.array([0, 0,  0, 1, -1], dtype=np.int_)
    wt = np.array([2/6, 1/6, 1/6, 1/6, 1/6])   


    ex_3d = ex.reshape(Q, 1, 1)
    ey_3d = ey.reshape(Q, 1, 1)
    wt_3d = wt.reshape(Q, 1, 1) 
    eu   =  ex_3d * ux[np.newaxis, :, :] + ey_3d * uy[np.newaxis, :, :]
    eed  = ((ex_3d - ux[np.newaxis, :, :]) * dTdx[np.newaxis, :, :] + (ey_3d - uy[np.newaxis, :, :]) * dTdy[np.newaxis, :, :])

    feq[:, :, :]= wt_3d *T*(1 + eu / cs2)    
    f[:, :, :] = feq[:, :, :] - (taut-1)*dt*eed*wt_3d *(1 + eu / cs2)
    fneq[:, :, :] = f[:, :, :] - feq[:, :, :]


    f[3, 0, :]  = feq[3,  0, :] + fneq[3,  1, :]
    f[4,-1, :]  = feq[4, -1, :] + fneq[4, -2, :]
    f[1, :, 0]  = feq[1,  :, 0] + fneq[1,  :, 1] 
    f[2, :,-1]  = feq[2,  :,-1] + fneq[2,  :,-2]       
  
    return f.flatten()

In [ ]:
def vorCirc(rho,vor,ux,uy,dvordx,dvordy):
    qTOTAL=Qexlink+nlinks
    q = QuantumRegister(qTOTAL,'q')

    setup = QuantumCircuit(q)
    
    A_diag = vorf(rho,vor,ux,uy,dvordx,dvordy)
       
    zeros = np.zeros(Ny*Nx)
    A_diag = np.concatenate((A_diag, np.zeros(2**(qTOTAL-1) - A_diag.size)))
    B1_diag = top_half(A_diag)
    B2_diag = bottom_half(A_diag)
    
    setup.h(qTOTAL-1)
    
    Col1 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col1.diagonal(list(B1_diag), qubit=list(range(qTOTAL - 1)))
    Col1 = Col1.to_gate(label='c1')
    
    Col2 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col2.diagonal(list(B2_diag),qubit = list(range(qTOTAL - 1)))
    Col2 = Col2.to_gate(label='c2')
    
    target_qubits = list(range(qTOTAL-1))      
    full_qubits = [qTOTAL-1] + target_qubits   

    setup.append(Col1.control(1, ctrl_state='0'), full_qubits)
    setup.append(Col2.control(1, ctrl_state='1'), full_qubits)

    setup.h(qTOTAL-1)

    setup.barrier()
    L1 = lshift(nlatx).to_gate(label = "1L").control (nlinks,ctrl_state = '001')#1
    R2 = rshift(nlatx).to_gate(label = "2R").control (nlinks,ctrl_state = '010')#2

    L3 = lshift(nlaty).to_gate(label = "3LT").control(nlinks,ctrl_state = '011')#3
    R4 = rshift(nlaty).to_gate(label = "4RT").control(nlinks,ctrl_state = '100')#4

    setup.append(L1,nlinks1 + list(reversed(nlatx1)))
    setup.append(R2,nlinks1 + list(reversed(nlatx1)))
    setup.append(L3,nlinks1 + list(reversed(nlaty1)))
    setup.append(R4,nlinks1 + list(reversed(nlaty1)))  
    setup.barrier()

    for i in range(nlinks): 
        setup.h(qTOTAL-2-i)

    return setup

vorCirc(rho,rho,ux,uy,dd,dd).draw()

In [ ]:

def phiCirc(rho,phi,ux,uy):
    qTOTAL=Qexlink+nlinks
    q = QuantumRegister(qTOTAL,'q')

    setup = QuantumCircuit(q)
    
    A_diag = phif(rho,vor,ux,uy)
       
    zeros = np.zeros(Ny*Nx)
    A_diag = np.concatenate((A_diag, np.zeros(2**(qTOTAL-1) - A_diag.size)))
    B1_diag = top_half(A_diag)
    B2_diag = bottom_half(A_diag)
    
    setup.h(qTOTAL-1)
    
    Col1 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col1.diagonal(list(B1_diag), qubit=list(range(qTOTAL - 1)))
    Col1 = Col1.to_gate(label='c1')
    
    Col2 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col2.diagonal(list(B2_diag),qubit = list(range(qTOTAL - 1)))
    Col2 = Col2.to_gate(label='c2')
    
    target_qubits = list(range(qTOTAL-1))       
    full_qubits = [qTOTAL-1] + target_qubits    

    setup.append(Col1.control(1, ctrl_state='0'), full_qubits)
    setup.append(Col2.control(1, ctrl_state='1'), full_qubits)

    setup.h(qTOTAL-1)

    setup.barrier()
    L1 = lshift(nlatx).to_gate(label = "1L").control (nlinks,ctrl_state = '001')
    R2 = rshift(nlatx).to_gate(label = "2R").control (nlinks,ctrl_state = '010')

    L3 = lshift(nlaty).to_gate(label = "3LT").control(nlinks,ctrl_state = '011')
    R4 = rshift(nlaty).to_gate(label = "4RT").control(nlinks,ctrl_state = '100')

    setup.append(L1,nlinks1 + list(reversed(nlatx1)))
    setup.append(R2,nlinks1 + list(reversed(nlatx1)))
    setup.append(L3,nlinks1 + list(reversed(nlaty1)))
    setup.append(R4,nlinks1 + list(reversed(nlaty1)))  
    setup.barrier()

    for i in range(nlinks): 
        setup.h(qTOTAL-2-i)

    return setup

phiCirc(rho,rho,ux,uy).draw()

In [ ]:

def TCirc(rho,T,ux,uy,dTdx,dTdy):
    qTOTAL=Qexlink+nlinks
    q = QuantumRegister(qTOTAL,'q')

    setup = QuantumCircuit(q)
    
    A_diag = Tf(rho,T,ux,uy,dTdx,dTdy)
       
    zeros = np.zeros(Ny*Nx)
    A_diag = np.concatenate((A_diag, np.zeros(2**(qTOTAL-1) - A_diag.size)))
    B1_diag = top_half(A_diag)
    B2_diag = bottom_half(A_diag)
    
    setup.h(qTOTAL-1)
    
    Col1 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col1.diagonal(list(B1_diag), qubit=list(range(qTOTAL - 1)))
    Col1 = Col1.to_gate(label='c1')
    
    Col2 = QuantumCircuit(QuantumRegister(qTOTAL-1))
    Col2.diagonal(list(B2_diag),qubit = list(range(qTOTAL - 1)))
    Col2 = Col2.to_gate(label='c2')
    
    target_qubits = list(range(qTOTAL-1))        
    full_qubits = [qTOTAL-1] + target_qubits     

    setup.append(Col1.control(1, ctrl_state='0'), full_qubits)
    setup.append(Col2.control(1, ctrl_state='1'), full_qubits)

    setup.h(qTOTAL-1)

    setup.barrier()
    L1 = lshift(nlatx).to_gate(label = "1L").control (nlinks,ctrl_state = '001')
    R2 = rshift(nlatx).to_gate(label = "2R").control (nlinks,ctrl_state = '010')

    L3 = lshift(nlaty).to_gate(label = "3LT").control(nlinks,ctrl_state = '011')
    R4 = rshift(nlaty).to_gate(label = "4RT").control(nlinks,ctrl_state = '100')

    setup.append(L1,nlinks1 + list(reversed(nlatx1)))
    setup.append(R2,nlinks1 + list(reversed(nlatx1)))
    setup.append(L3,nlinks1 + list(reversed(nlaty1)))
    setup.append(R4,nlinks1 + list(reversed(nlaty1)))  
    setup.barrier()

    for i in range(nlinks): 
        setup.h(qTOTAL-2-i)

    return setup
TCirc(rho,rho,ux,uy,dd,dd).draw()

In [ ]:
def vortimestep(rho,vor,ux,uy,dvordx,dvordy):   
    zeros = np.zeros((Ny,Nx))
    vors = np.concatenate((rho,rho,rho,rho,rho,zeros,zeros,zeros)).flatten()    
    vorSV = Statevector(vors).expand([1,0]).evolve(vorCirc(rho,vor,ux,uy,dvordx,dvordy))
    vorAr = np.reshape(np.asarray(vorSV)[:Ny*Nx],(Ny,Nx))
    vorAr = np.real(vorAr)*2**(nlinks/2)

    return np.reshape(vorAr,(Ny,Nx))


def phitimestep(rho,phi,ux,uy):
    zeros = np.zeros((Ny,Nx))
    phis = np.concatenate((rho,rho,rho,rho,rho,zeros,zeros,zeros)).flatten() 
    phiSV = Statevector(phis).expand([1,0]).evolve(phiCirc(rho,phi,ux,uy))
    phiAr = np.reshape(np.asarray(phiSV)[:Ny*Nx],(Ny,Nx))
    phiAr = np.real(phiAr)*2**(nlinks/2)

    return np.reshape(phiAr,(Ny,Nx))


def Ttimestep(rho,T,ux,uy,dTdx,dTdy):
    
    zeros = np.zeros((Ny,Nx))
    tf =  np.concatenate((rho,rho,rho,rho,rho,zeros,zeros,zeros)).flatten()
    tfSV = Statevector(tf).expand([1,0]).evolve(TCirc(rho,T,ux,uy,dTdx,dTdy))
    tfAr = np.reshape(np.asarray(tfSV)[:Ny*Nx],(Ny,Nx))
    tfAr = np.real(tfAr)*2**(3/2)
    
    return np.reshape(tfAr,(Ny,Nx))    

In [ ]:
def visualize(n, allUx, allUy, phi, T):
    density=1
    x = np.linspace(0, L, Nx)
    y = np.linspace(0, L, Ny)
    X, Y = np.meshgrid(x, y)
    tick_positions = [0, 2.5, 5, 7.5, 10, 12.5, 15]


    fig, ax = plt.subplots()


    ax.quiver(X[::density, ::density], Y[::density, ::density], 
                  allUx[n+1][::density, ::density], allUy[n+1][::density, ::density], 
                  scale=50)  
    
    
    ax.set_xlim(0, L)  
    ax.set_ylim(0, L)  


    ax.set_xticks(tick_positions)  
    ax.set_yticks(tick_positions)  
    
  
    ax.set_title(f'Vector Field at t={n + 1}')
    ax.set_aspect('equal', adjustable='box')

    plt.savefig(f'ggg1000/vector{n + 1}.png', bbox_inches='tight')
    plt.close(fig) 
    fig, ax = plt.subplots()
    CS = ax.contour(np.arange(Nx), np.arange(Ny), phi, levels=20)
    ax.clabel(CS, inline=True, fontsize=10)
    ax.set_title(f't={n+1}')
    ax.set_aspect('equal', adjustable='box')
    
    CS = ax.contour(X, Y, phi, levels=20)
    ax.clabel(CS, inline=True, fontsize=10)
    ax.set_title(f't={n+1}')
    ax.set_aspect('equal', adjustable='box')
   
    plt.savefig(f'ggg1000/contourphi{n+1}.png', bbox_inches='tight')
    plt.close(fig) 

    fig, ax = plt.subplots()
    CS = ax.contour(np.arange(Nx), np.arange(Ny), T, levels=20)
    ax.clabel(CS, inline=True, fontsize=10)
    ax.set_title(f't={n+1}')
    ax.set_aspect('equal', adjustable='box')
    
    CS = ax.contour(X, Y, T, levels=20)
    ax.clabel(CS, inline=True, fontsize=10)
    ax.set_title(f't={n+1}')
    ax.set_aspect('equal', adjustable='box')
  
    plt.savefig(f'ggg1000/contourT{n+1}.png', bbox_inches='tight')
    plt.close(fig)  


In [ ]:
allrho = [rho.copy()]
allvor = [vor.copy()]
allphi = [phi.copy()]
allux = [ux.copy()]  
alluy = [uy.copy()]
allUx = [Ux.copy()]  
allUy = [Uy.copy()]
allT = [T.copy()]
allNu = [Nu.copy()]

In [ ]:
steps = 10000

for n in range(steps):

    ux,uy = allux[-1], alluy[-1]
    rho = allrho[-1]
    vor, phi,  = allvor[-1], allphi[-1]
    Ux, Uy = allUx[-1], allUy[-1]
    T, Nu = allT[-1], allNu[-1]

    Tm = np.sum(T)/Nx/Ny

    dvordx,dvordy,dTdx,dTdy = graduv(vor,T)

    vor  = vortimestep(rho,vor,ux,uy,dvordx,dvordy)
    vor = vor -  dt*U*U*Ra*dTdx/Pr/Re/Re

    phi  = phitimestep(rho,phi,ux,uy)
    phi = phi + dt*vor
    phi,vor = pvbounds(phi) 

    ux =  np.gradient(phi, axis=0) 
    uy = -np.gradient(phi, axis=1) 

    ux,uy = UVbounds()    

    T = Ttimestep(rho,T,ux,uy,dTdx,dTdy)
    T = Tbounds()

    Ux=ux/U
    Uy=uy/U

    allrho.append(rho.copy())
    allvor.append(vor.copy())    
    allphi.append(phi.copy())              
    allux.append(ux.copy())  
    alluy.append(uy.copy())  
    allUx.append(Ux.copy())  
    allUy.append(Uy.copy())  
    allT.append(T.copy())  
    allNu.append(Nu.copy())

    if (n + 1) % 5 == 0:
        np.savetxt(f'ggg1000/u_gradient_t{n+1}.csv', Ux, delimiter=',')
        np.savetxt(f'ggg1000/v_gradient_t{n+1}.csv', Uy, delimiter=',')    
        visualize(n, allUx, allUy, phi, T)
